**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-349a_zcn
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-349a_zcn
  Resolved https://github.com/Kaggle/learntools.git to commit 2f2108425302b9a9e63e5c9040e8484437ce9737
  Preparing metadata (setup.py) ... - done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268981 sha256=b3d7a35a6b5362020dd70d53f6b87b6b257094c984b54ba81e505d59a02db501
  Stored in directory: /tmp/pip-ephem-wheel-cache-pdx9bwkk/wheels/dd/d7/6b/0fc758f52767fd281d6dceded6757c6cb5bb90ccd2dbb1de9f
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,693dab0bda261ca69e99b9a41ff5693b99ed6e60,4eefd6d2716e3e70208114acab05b879317f0277c8a8e6...,2018-06-25 08:00:00+00:00,2018-06-25 08:00:00+00:00,299,0.5,1.703128e+10,1.703184e+10,28,32,...,0.0,6.25,Credit Card,Yellow Cab,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707)
1,c8a823094d66133d9cfbc93c4014e7063108e5fe,397fda5f3737a7b1a8ee587706ebb7e2ba198216728b26...,2018-07-12 13:00:00+00:00,2018-07-12 13:00:00+00:00,534,1.1,1.703132e+10,1.703108e+10,32,8,...,2.5,9.50,Cash,Yellow Cab,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134)
2,4e582997db30a0c6888cc1c249635c9e346d1532,57b6a83a3d58fb553e327e9f07e7de0d4f81be215174f3...,2018-07-09 11:15:00+00:00,2018-07-09 11:30:00+00:00,641,1.7,1.703128e+10,1.703108e+10,28,8,...,0.0,10.50,Credit Card,Yellow Cab,41.885300,-87.642808,POINT (-87.6428084655 41.8853000224),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134)
3,e987e78040efc1b8a2d748acaae430ed92683dc0,922a0223b6c4173a158b0f1fed098dd8c6e2a303f520b2...,2018-04-07 14:15:00+00:00,2018-04-07 14:15:00+00:00,760,2.4,1.703184e+10,1.703133e+10,32,33,...,2.0,12.25,Cash,Checker Taxi,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.859350,-87.617358,POINT (-87.6173580061 41.859349715)
4,912c7a0ceb30d2a4d8370ba8144bc69a7e37becd,7f7b8460175e2fafd1ea35903c8c8482d76dd4e610e200...,2018-07-15 16:30:00+00:00,2018-07-15 16:45:00+00:00,1632,9.3,NaN,NaN,8,41,...,2.0,33.93,Credit Card,Yellow Cab,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.794090,-87.592311,POINT (-87.592310855 41.794090253)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,trip_date,avg_num_trips
0,2016-01-31,80539.000000
1,2016-01-10,82083.428571
2,2016-02-25,91758.857143
3,2016-02-28,92816.428571
4,2016-01-18,85136.000000


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,pickup_community_area,trip_start_timestamp,trip_end_timestamp
0,NaN,2013-10-03 06:15:00+00:00,2013-10-03 06:30:00+00:00
1,NaN,2013-10-03 00:45:00+00:00,2013-10-03 01:15:00+00:00
2,NaN,2013-10-03 03:15:00+00:00,2013-10-03 03:15:00+00:00
3,NaN,2013-10-03 04:00:00+00:00,2013-10-03 04:00:00+00:00
4,NaN,2013-10-03 05:00:00+00:00,2013-10-03 05:15:00+00:00


<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> There are many different numbering functions that enumerate the rows in the input. For this exercise, please use the **RANK()** function.

In [6]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/zzHgmlx.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           ____ 
                               OVER (
                                    PARTITION BY ____ 
                                    ORDER BY ____), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> You don't have a valid query yet.  Try again.

In [8]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*